# Part2-1 CRF

In [1]:
token_ids = []
label_ids = []

from torch.utils.data import Dataset, DataLoader

class CWS_dataset(Dataset):
    def __init__(self, data_path, tokenizer):
        
        self.tokenizer = tokenizer
        self.label_map = {label: i for i, label in enumerate(['B','M','E','S','O'])}
        
        with open(data_path , 'r') as f:
            data = f.readlines()
            
            sentence, labels = [], []

            string, output = ['[CLS]'], [4]
            for w in data:
                if w != '\n':
                    string.append(w[:w.find("\t")])
                    output.append(self.label_map[w[-2]])
                elif w == '\n':
                    sentence.append(string+['[SEP]']), labels.append(output+[4])
                    string, output = ['[CLS]'], [4]
                                
            self.data = {'sentence':sentence, 'labels':labels}
            assert len(self.data['sentence'])==len(self.data['labels']) 
    
    def __len__(self):
        assert len(self.data['sentence'])==len(self.data['labels']) 
        return len(self.data['sentence'])
    
    def __getitem__(self, idx):
        return self.data['sentence'][idx], self.data['labels'][idx]
    
    def collate_fn(self, samples):
        tokenizer = self.tokenizer
        
        b_token_ids = []
        b_label_ids = []
        
        ## word to token id
        for sentence, labels in samples:
            token_ids, label_ids = [], []
            for index, word in enumerate(sentence):
                ids = tokenizer.convert_tokens_to_ids(word)
                
                token_ids.append(ids)
                label_ids.append(labels[index])
                
            assert len(token_ids)==len(label_ids)
                
            b_token_ids.append(token_ids)
            b_label_ids.append(label_ids)
            
        ## fix to same length 
        max_length = max([len(token_ids) for token_ids in b_token_ids])
        max_length = 512 if max_length>512 else max_length
        
        PAD_token = tokenizer.convert_tokens_to_ids('[PAD]')
        for token_ids, label_ids in zip(b_token_ids, b_label_ids):
            token_ids += [PAD_token]*(max_length-len(token_ids))
            label_ids += [4]*(max_length-len(label_ids))
        
        return torch.tensor(b_token_ids), torch.tensor(b_label_ids)

In [2]:

# import torch

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
# model_config = BertConfig(vocab_size=tokenizer.vocab_size, hidden_size=768, num_hidden_layers=4, num_labels=5)
# #model = BertForTokenClassification(config=model_config)
# model = BertForTokenClassification.from_pretrained('bert-base-chinese', num_labels=5)
# model.config.save_pretrained('output')  #save config

In [3]:
import torch
import torch.nn as nn

def metrics(output, label):
    output = torch.max(output, 1)[1]
    acc = ((output==label).int()*(label!=4).int()).sum().item()/((label>=0).int()*(label!=4).int()).sum().item()
    return acc

def _run_train(args, model, criterion, optimizer, dataloader):
    
    model.train()
    
    total_loss, total_acc = 0, 0
    for idx, (b_token_ids, b_label_ids) in enumerate(dataloader):
        b = b_token_ids.shape[0]
                
        output = model(b_token_ids.cuda())[0]
        
        output = output.view(-1,5)
        b_label_ids = b_label_ids.cuda().view(-1)
        
        ## calcu loss
        optimizer.zero_grad()
        loss = criterion(output, b_label_ids)
        loss.backward()        
        optimizer.step()
        
        total_loss += loss.item()*b
        total_acc += metrics(output, b_label_ids)*b
        print("\t[{}/{}] train loss:{:.3f}, acc:{:.3f} ".format(
                                            idx+1,
                                            len(dataloader),
                                            total_loss/(idx+1)/b,
                                            total_acc/(idx+1)/b),
                                    end='   \r')     

    return total_loss/len(dataloader.dataset), total_acc/len(dataloader.dataset)

def _run_valid(args, model, criterion, dataloader):
    
    model.eval()
    
    with torch.no_grad():
    
        total_loss, total_acc = 0, 0
        for idx, (b_token_ids, b_label_ids) in enumerate(dataloader):
            b = b_token_ids.shape[0]

            output = model(b_token_ids.cuda())[0]

            output = output.view(-1,5)
            b_label_ids = b_label_ids.cuda().view(-1)

            ## calcu loss
            loss = criterion(output, b_label_ids)

            total_loss += loss.item()*b
            total_acc += metrics(output, b_label_ids)*b
            print("\t[{}/{}] valid loss:{:.3f}, acc:{:.3f} ".format(
                                                idx+1,
                                                len(dataloader),
                                                total_loss/(idx+1)/b,
                                                total_acc/(idx+1)/b),
                                        end='   \r')     

        return total_loss/len(dataloader.dataset), total_acc/len(dataloader.dataset) 

def train(args, train_dataloader, valid_dataloader):
    torch.manual_seed(987)
    torch.cuda.manual_seed(987)
    
    #if os.path.isfile(args.load_path):
        #model.load_state_dict(torch.load(args.load_model)['state_dict'])
    model.cuda() 
    
    criterion = nn.CrossEntropyLoss(ignore_index=4).cuda()
     
    optimizer = torch.optim.AdamW(list(model.parameters()), 
                                  lr  = args.lr,
                                  eps = 1e-8 )
    
    best_loss = 100
    for epoch in range(1,args.epoch+1):
        print(f' Epoch {epoch}')
            
        loss, acc = _run_train(args, model, criterion, optimizer, train_dataloader)
        print("\t[Info] Train avg loss:{:.4f}, acc:{:.4f}".format(loss,acc))
        
        loss, acc = _run_valid(args, model, criterion, valid_dataloader)
        print("\t[Info] Valid avg loss:{:.4f}, acc:{:.4f}".format(loss,acc))
        
        ## Save best model
        if loss < best_loss:
            #torch.save({'state_dict': model.state_dict()}, "{}/model_best.pt".format(args.save_path))
            model.save_pretrained("{}/".format(args.save_path))
            print('\t[Info] save weights best')
        ## Save newest model
        #torch.save({'state_dict': model.state_dict()}, "{}/model.pt".format(args.save_path))
        model.save_pretrained("{}/".format(args.save_path))
        print('\t[Info] save weights {}epoch'.format(epoch))
      
        ## Update learning rate
        '''
        if epoch>2:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 3
                if param_group['lr'] < 1e-6:
                    param_group['lr'] = 1e-6 
        '''
        print('\t--------------------------------------------------------')
    

In [4]:
import os, argparse

from transformers import BertTokenizer, BertForTokenClassification, BertConfig

def parse_args(string=""):
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', default=2e-5,
                            type=float, help='leanring rate')
    parser.add_argument('--epoch', default=3,
                            type=int, help='epochs')
    parser.add_argument('--batch-size', default=8,
                            type=int, help='batch size')
    parser.add_argument('--gpu', default="0",
                            type=str, help="0:1080ti 1:1070")
    parser.add_argument('--num-workers', default=8,
                            type=int, help='dataloader num workers')
    parser.add_argument('--save-path', default='OUT_DIR',
                            type=str, help='.pth model file save dir')
    parser.add_argument('--load-path', default='trained_model/my_cws_bert.pt',
                            type=str, help='.pth model file save dir')
    args = parser.parse_args(string)
    return args

if __name__=='__main__':
    args = parse_args()
    
    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu #0:1080ti 1:1070
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    
    #model_config = BertConfig(vocab_size=tokenizer.vocab_size, hidden_size=768, num_hidden_layers=4, num_labels=5)
    #model = BertForTokenClassification(config=model_config)
    model = BertForTokenClassification.from_pretrained('bert-base-chinese', num_labels=5)
    model.config.save_pretrained(args.save_path)  #save config

    train_dataset = CWS_dataset('/media/D/DLHLP/hw4/CWS_dlhlp/train_proc.txt', tokenizer)
    valid_dataset = CWS_dataset('/media/D/DLHLP/hw4/CWS_dlhlp/dev_proc.txt', tokenizer)

    train_dataloader = DataLoader(train_dataset, 
                                  batch_size = args.batch_size,
                                  num_workers= args.num_workers,                              
                                  collate_fn = train_dataset.collate_fn,
                                  shuffle = True)
    valid_dataloader = DataLoader(train_dataset, 
                                  batch_size = args.batch_size*3,
                                  num_workers= args.num_workers,
                                  collate_fn = train_dataset.collate_fn)

    train(args, train_dataloader, valid_dataloader)
    
    print("finish~")

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.1.1) doesn't match a supported version!
  RequestsDependencyWarning)


 Epoch 1
	[Info] Train avg loss:0.0957, acc:0.9672   
	[Info] Valid avg loss:0.0343, acc:0.9881 
	[Info] save weights best
	[Info] save weights 1epoch
	--------------------------------------------------------
 Epoch 2
	[Info] Train avg loss:0.0419, acc:0.9854   
	[Info] Valid avg loss:0.0202, acc:0.9931 
	[Info] save weights best
	[Info] save weights 2epoch
	--------------------------------------------------------
 Epoch 3
	[Info] Train avg loss:0.0257, acc:0.9909   
	[Info] Valid avg loss:0.0127, acc:0.9958 
	[Info] save weights best
	[Info] save weights 3epoch
	--------------------------------------------------------
finish~


## Result


In [2]:
import torch

from transformers import BertTokenizer, BertForTokenClassification, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
config = BertConfig.from_pretrained('OUT_DIR/config.json')
model = BertForTokenClassification.from_pretrained('OUT_DIR/pytorch_model.bin', config=config).cuda()#, config=config).cuda()

report_sentence = ['我一直親自指揮、親自部署，我相信只要我們堅定信心、同舟共濟、科學防治、精準施策，我們一定會戰勝這一次疫情。',
                   '這個聲明讓我再次想起了安徒生的童話《皇帝的新裝》。',
                   '希望他們能夠聽一聽這個忠告，不要再信口雌黃地抹黑，居心叵測地挑撥，煞有介事地恫嚇。',
                   '有關部門當然就是有關的部門了。無關的就不能稱為有關部門。所以我建議你還是要向他們詢問。',
                   '不要搞奇奇怪怪的建築。',
                   '現在提請表決。同意的代表請舉手。請放下；不同意的請舉手。沒有；棄權的請舉手。沒有。通過！',
                   '輕關易道，通商寬衣。',
                   '人均國內生產總值接近八千萬美元。',
                   '我青年時代就對法國文化抱有濃厚興趣，法國的歷史、哲學、文學、藝術深深吸引著我。讀法國近現代史特別是法國大革命史的書籍，讓我豐富了對人類社會政治演進規律的思考。讀孟德斯鳩、伏爾泰、盧梭、狄德羅、聖西門、傅立葉、薩特等人的著作，讓我加深了對思想進步對人類社會進步作用的認識。讀蒙田、拉封丹、莫里哀、司湯達、巴爾扎克、雨果、大仲馬、喬治·桑、福樓拜、小仲馬、莫泊桑、羅曼·羅蘭等人的著作，讓我增加了對人類生活中悲歡離合的感觸。冉阿讓、卡西莫多、羊脂球等藝術形象至今仍栩栩如生地存在於我的腦海之中。欣賞米勒、馬奈、德加、塞尚、莫內、羅丹等人的藝術作品，以及趙無極中西合璧的畫作，讓我提升了自己的藝術鑑賞能力。還有，讀凡爾納的科幻小說，讓我的頭腦充滿了無盡的想像。',
                   '因為我那時候，扛兩百斤麥子，十里山路不換肩的。']

rm_notate = True
if rm_notate:
    for idx,sent in enumerate(report_sentence):
        for c in "，、!。！。;；《》":
            sent = sent.replace(c,"")
        report_sentence[idx] = sent

label_map = {label: i for i, label in enumerate(['B','M','E','S','O'])}

output_sentence = []

for sentence in report_sentence:

    # 標記
    token_ids = tokenizer.encode(sentence)

    model.eval()
    label = model(torch.tensor([token_ids]).cuda())[0]
    label = torch.max(label[:,1:-1],2)[1].cpu().numpy().tolist()[0]
    
    # 斷詞
    assert len(sentence)==len(label)
    seg_sentence = ''
    for l,w in zip(label,sentence):
        if l==label_map['B']:
            seg_sentence += f'{w}'
        elif l==label_map['E']:
            seg_sentence += f'{w},'
        elif l==label_map['M']:
            seg_sentence += f'{w}'
        elif l==label_map['S']:
            seg_sentence += f'{w},'
        
    output_sentence.append(seg_sentence[:-1])

for sent in output_sentence:
    print(sent)
    continue
    
with open('segmented.csv', 'w') as f:
    for sent in output_sentence:
        f.write(sent)
        f.write('\n')


我,一直,親自,指揮,親自,部署,我,相信,只要,我們,堅定,信心,同舟共濟,科學,防治,精準,施策,我們,一定,會,戰勝,這,一,次,疫情
這,個,聲明,讓,我,再次,想起,了,安徒生,的,童話,皇帝,的,新裝
希望,他們,能夠,聽,一,聽,這,個,忠告,不,要,再,信口雌黃,地,抹黑,居心叵測,地,挑撥,煞有介事,地,恫嚇
有關,部門,當然,就,是,有關,的,部門,了,無關,的,就,不,能,稱為,有關,部門,所以,我,建議,你,還是,要,向,他們,詢問
不,要,搞,奇奇怪怪,的,建築
現在,提請,表決,同意,的,代表,請,舉手,請,放下,不,同意,的,請,舉手,沒有,棄權,的,請,舉手,沒有,通過
輕關,易道,通商,寬衣
人均,國內,生產,總值,接近,八千萬,美元
我,青年,時代,就,對,法國,文化,抱有,濃厚,興趣,法國,的,歷史,哲學,文學,藝術,深深,吸引,著,我,讀,法國,近現代史,特別是,法國,大,革命史,的,書籍,讓,我,豐富,了,對,人類,社會,政治,演進,規律,的,思考,讀,孟德斯鳩伏爾泰盧梭狄德羅,聖西門傅立葉薩特,等,人,的,著作,讓,我,加深,了,對,思想,進步,對,人類,社會,進步,作用,的,認識,讀,蒙田拉封丹莫里哀司湯達巴爾扎克,雨果,大仲馬,喬治·桑福樓拜,小,仲馬,莫泊桑羅曼·羅蘭,等,人,的,著作,讓,我,增加,了,對,人類,生活,中,悲歡離合,的,感觸,冉阿讓卡西莫多,羊脂球,等,藝術,形象,至今,仍,栩栩如生,地,存在,於,我,的,腦海,之中,欣賞,米勒馬奈德加塞尚莫內羅丹,等,人,的,藝術,作品,以及,趙無極,中,西,合璧,的,畫作,讓,我,提升,了,自己,的,藝術,鑑賞,能力,還有,讀,凡爾納,的,科幻,小說,讓,我,的,頭腦,充滿,了,無盡,的,想像
因為,我,那,時候,扛,兩百,斤,麥子,十,里,山路,不,換肩,的


---
---

## with notation
我,一直,親自,指揮,、,親自,部署,，,我,相信,只要,我們,堅定,信心,、,同舟共濟,、,科學,防治,、,精準,施策,，,我們,一定,會,戰勝,這,一,次,疫情,。
這,個,聲明,讓,我,再次,想起,了,安徒生,的,童話,《,皇帝,的,新裝,》,。
希望,他們,能夠,聽,一,聽,這,個,忠告,，,不,要,再,信口雌黃,地,抹黑,，,居心叵測,地,挑撥,，,煞有介事,地,恫嚇,。
有關,部門,當然,就,是,有關,的,部門,了,。,無關,的,就,不,能,稱為,有關,部門,。,所以,我,建議,你,還是,要,向,他們,詢問,。
不,要,搞,奇奇怪怪,的,建築,。
現在,提請,表決,。,同意,的,代表,請,舉手,。,請,放下,；,不,同意,的,請,舉手,。,沒有,；,棄權,的,請,舉手,。,沒有,。,通過,！
輕關易道,，,通商,寬衣,。
人均,國內,生產,總值,接近,八千萬,美元,。
我,青年,時代,就,對,法國,文化,抱有,濃厚,興趣,，,法國,的,歷史,、,哲學,、,文學,、,藝術,深深,吸引,著,我,。,讀,法國,近現代史,特別是,法國,大,革命史,的,書籍,，,讓,我,豐富,了,對,人類,社會,政治,演進,規律,的,思考,。,讀,孟德斯鳩,、,伏爾泰,、,盧梭,、,狄德羅,、,聖西門,、,傅立葉,、,薩特,等,人,的,著作,，,讓,我,加深,了,對,思想,進步,對,人類,社會,進步,作用,的,認識,。,讀,蒙田,、,拉封丹,、,莫里哀,、,司湯達,、,巴爾扎克,、,雨果,、,大仲馬,、,喬治·桑,、,福樓拜,、,小仲馬,、,莫泊桑,、,羅曼·羅蘭,等,人,的,著作,，,讓,我,增加,了,對,人類,生活,中,悲歡離合,的,感觸,。,冉阿讓,、,卡西莫多,、,羊脂球,等,藝術,形象,至今,仍,栩栩如生,地,存在,於,我,的,腦海,之中,。,欣賞,米勒,、,馬奈,、,德加,、,塞尚,、,莫內,、,羅丹,等,人,的,藝術,作品,，,以及,趙無極,中,西,合璧,的,畫作,，,讓,我,提升,了,自己,的,藝術,鑑賞,能力,。,還有,，,讀,凡爾納,的,科幻,小說,，,讓,我,的,頭腦,充滿,了,無盡,的,想像,。
因為,我,那,時候,，,扛,兩百,斤,麥子,，,十里山路,不,換肩,的,。

## without notation
我,一直,親自,指揮,親自,部署,我,相信,只要,我們,堅定,信心,同舟共濟,科學,防治,精準,施策,我們,一定,會,戰勝,這,一,次,疫情
這,個,聲明,讓,我,再次,想起,了,安徒生,的,童話,《,皇帝,的,新裝,》
希望,他們,能夠,聽,一,聽,這,個,忠告,不,要,再,信口雌黃,地,抹黑,居心叵測,地,挑撥,煞有介事,地,恫嚇
有關,部門,當然,就,是,有關,的,部門,了,無關,的,就,不,能,稱為,有關,部門,所以,我,建議,你,還是,要,向,他們,詢問
不,要,搞,奇奇怪怪,的,建築
現在,提請,表決,同意,的,代表,請,舉手,請,放下,；,不,同意,的,請,舉手,沒有,；,棄權,的,請,舉手,沒有,通過,！
輕關,易道,通商,寬衣
人均,國內,生產,總值,接近,八千萬,美元
我,青年,時代,就,對,法國,文化,抱有,濃厚,興趣,法國,的,歷史,哲學,文學,藝術,深深,吸引,著,我,讀,法國,近現代史,特別是,法國,大,革命史,的,書籍,讓,我,豐富,了,對,人類,社會,政治,演進,規律,的,思考,讀,孟德斯鳩,伏爾泰盧梭狄德羅聖西門傅立葉薩特,等,人,的,著作,讓,我,加深,了,對,思想,進步,對,人類,社會,進步,作用,的,認識,讀,蒙田拉封丹,莫里哀司湯達巴爾扎克,雨果,大仲馬,喬治·桑福樓拜,小仲馬,莫泊桑羅曼·羅蘭,等,人,的,著作,讓,我,增加,了,對,人類,生活,中,悲歡離合,的,感觸,冉阿讓卡西莫多,羊脂球,等,藝術,形象,至今,仍,栩栩如生,地,存在,於,我,的,腦海,之中,欣賞,米勒馬奈德加塞尚莫內羅丹,等,人,的,藝術,作品,以及,趙無極,中,西,合璧,的,畫作,讓,我,提升,了,自己,的,藝術,鑑賞,能力,還有,讀,凡爾納,的,科幻,小說,讓,我,的,頭腦,充滿,了,無盡,的,想像
因為,我,那,時候,扛,兩百,斤,麥子,十里山路,不,換肩,的